### Reward Model
This notebook constructs the Reward Model that will be used in a PPO step at a later stage. Still takes the dilbert uncased and builds on top of it

In [1]:
import random
from pathlib import Path
from datasets import Dataset
from shared_models import HellaSwagEntry
from transformers import (AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments)

from peft import LoraConfig, get_peft_model

import torch
import evaluate
import numpy as np

#### Data Collection

In [2]:
DATA_PATH = Path("../data/hellaswag_format/personal_chat_sessions_train_hellaswag.jsonl")

In [3]:
def load_jsonl_pydantic(path):
    """Yield HellaSwagEntry objects parsed with Pydantic."""
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            yield HellaSwagEntry.model_validate_json(line)

# Build pairwise examples
pairs = []
for ex in load_jsonl_pydantic(DATA_PATH):
    endings = [ex.ending0, ex.ending1, ex.ending2, ex.ending3, ex.ending4]
    pos_id = ex.label
    neg_id = random.choice([i for i in range(5) if i != pos_id])

    pos_txt, neg_txt = endings[pos_id].strip(), endings[neg_id].strip()
    context = ex.context.strip()

    # randomly order A/B
    if random.random() < 0.5:
        first, second, lbl = pos_txt, neg_txt, 1
    else:
        first, second, lbl = neg_txt, pos_txt, 0

    pairs.append({
        "context": context,
        "first_resp": first,
        "second_resp": second,
        "label": lbl
    })

# Create HF Dataset
dataset = Dataset.from_list(pairs)
train_test = dataset.train_test_split(test_size=0.1, seed=42)

dataset

Dataset({
    features: ['context', 'first_resp', 'second_resp', 'label'],
    num_rows: 22282
})

#### Tokenization

In [4]:
model_ckpt = "google-bert/bert-base-uncased"
tokenizer  = AutoTokenizer.from_pretrained(model_ckpt)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
def tokenize_fn(examples):
    # Hugging Face will do: [CLS] context [SEP] first_resp [SEP] second_resp [SEP]
    tokenizerSnd= tokenizer(
        examples["context"],
        [f"{a} {tokenizer.sep_token} {b}" for a, b in zip(examples["first_resp"], examples["second_resp"])],
        truncation=True,
        max_length=128,
    )

    return tokenizerSnd

tokenized = train_test.map(tokenize_fn,
                           batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\aadhu\source\Anton\anton\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aadhu\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/20053 [00:00<?, ? examples/s]

Map:   0%|          | 0/2229 [00:00<?, ? examples/s]

#### Model LoRA Setup for Sequence Classification

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    print(f"\ntrainable model parameters: {trainable_model_params}\
    \nall model parameters: {all_model_params}\
    \npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%")

In [7]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=2
)

#Resize embeddings to match new tokenizer vocab
base_model.resize_token_embeddings(len(tokenizer))

# Model’s config should know about the pad token
base_model.config.pad_token_id = tokenizer.pad_token_id

for name, module in base_model.named_modules():
    print(name)
    print(module)

# Attach LoRA for parameter-efficient fine-tuning
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=4,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
)
model = get_peft_model(base_model, peft_config)

print_number_of_trainable_model_parameters(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

#### Training with Trainer

In [8]:
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

# TrainingArguments
args = TrainingArguments(
    output_dir="../data/models/reward_model_ckpts",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

C:\Users\aadhu\AppData\Local\Temp\ipykernel_14732\2318968774.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
print("Baseline:", trainer.evaluate())

Baseline: {'eval_loss': 0.7118020057678223, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.4791386271870794, 'eval_runtime': 132.0535, 'eval_samples_per_second': 16.88, 'eval_steps_per_second': 0.53}


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.695500,0.689722,0.005400,0.534769
2,0.402200,0.344743,0.005400,0.856438
3,0.359300,0.307014,0.005400,0.873934


TrainOutput(global_step=3762, training_loss=0.5303614736553964, metrics={'train_runtime': 7497.749, 'train_samples_per_second': 8.024, 'train_steps_per_second': 0.502, 'total_flos': 3964008332325888.0, 'train_loss': 0.5303614736553964, 'epoch': 3.0})

In [11]:
print("Final:", trainer.evaluate())

Final: {'eval_loss': 0.3070140779018402, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.8739344997756842, 'eval_runtime': 121.3309, 'eval_samples_per_second': 18.371, 'eval_steps_per_second': 0.577, 'epoch': 3.0}
